[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [3]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data_train =  pd.read_csv("./gdrive/My Drive/devC_data/labeledTrainData.tsv", sep='\t', encoding='latin-1')
data_train.sample()

,id,sentiment,review
21975,4788_10,1,This is one of the greatest films I have ever ...


In [4]:
data_train.sample(5).review.values

array(['\\Cypher\\" is a cleverly conceived story about industrial espionage set in America in the not too distant future. While thematically not complex, this film does offer many different perspectives about personal loyalty, ruthlessness, and corporate conspiracy. To a certain extent this film also attempts to represent modern corporate groups and companies as being indifferent to the risks their contract employees take on their behalf.<br /><br />The film starts off with a somewhat mediocre salary man, Morgan Sullivan (Jeremy Northam), who applies to the Digicorp group to work as an undercover operative. After an initial briefing with Digicorp\'s Security Chief, Sullivan is then given a new identity (Jack Thursby) and sent to a business conference with the task of recording the speeches given by various spokesmen concerning the marketing strategies of each of their respective companies. Upon successfully completing his first assignment, Sullivan/Thursby is sent on further missions 

In [5]:
from bs4 import BeautifulSoup

In [6]:
raw_review = data_train.review.values[0]
BeautifulSoup(raw_review, "lxml").get_text()

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

## 2. Preprocessing

In [7]:
# stop words
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = BeautifulSoup(text, "lxml").get_text()

    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    
    return text

In [9]:
data_train['processed_reviews'] = data_train.review.apply(preprocessor)

In [10]:
# split the dataset in train and test
# Your code here

X = data_train[['processed_reviews']]
y = data_train.sentiment.values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
y_train

array([1, 1, 1, ..., 1, 1, 0])

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=150000)

In [14]:
%%time
X_train_text = text_transformer.fit_transform(X_train['processed_reviews'])
X_test_text = text_transformer.transform(X_test['processed_reviews'])

CPU times: user 14.1 s, sys: 345 ms, total: 14.5 s
Wall time: 14.5 s


In [15]:
X_train_text

<17500x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 2166339 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import log_loss

lg = LogisticRegression()
lg.fit(X_train_text, y_train)
predictions = lg.predict(X_test_text)



## 4. Evaluate Model

In [17]:
# Show metrics
print("Accuracy score: %f" % accuracy_score(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print('Log loss:', log_loss(y_test, predictions)/len(y_test))

# Show parameters
print('w = ', lg.coef_)
print('b = ', lg.intercept_)


Accuracy score: 0.884533
Confusion Matrix:
[[3250  488]
 [ 378 3384]]
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      3738
           1       0.87      0.90      0.89      3762

    accuracy                           0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500

Log loss: 0.000531750587760686
w =  [[-0.08998597 -0.15358876  0.14132492 ... -0.03449163 -0.08202534
  -0.1134814 ]]
b =  [0.08870266]
